In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import time

#### Ejemplo MNIST

In [10]:
# Check if a GPU is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Define a simple neural network
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(784, 128)  # Input size: 28x28 = 784
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 10)   # Output size: 10 classes

    def forward(self, x):
        x = x.view(-1, 784)  # Flatten the input
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Load the MNIST dataset and apply transformations
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# Create an instance of the neural network
net = SimpleNN()
net.to(device)  # Move the network to the GPU

# Define a loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01)

# Training loop
for epoch in range(5):  # You can increase the number of epochs for better results
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:  # Print every 100 mini-batches
            print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 100:.3f}")
            running_loss = 0.0

print("Finished Training")

# Save the trained model
# torch.save(net.state_dict(), 'simple_nn.pth')


[1, 100] loss: 1.922
[1, 200] loss: 1.208
[1, 300] loss: 0.825
[1, 400] loss: 0.657
[1, 500] loss: 0.563
[1, 600] loss: 0.516
[1, 700] loss: 0.471
[1, 800] loss: 0.441
[1, 900] loss: 0.428
[2, 100] loss: 0.390
[2, 200] loss: 0.384
[2, 300] loss: 0.390
[2, 400] loss: 0.358
[2, 500] loss: 0.365
[2, 600] loss: 0.378
[2, 700] loss: 0.353
[2, 800] loss: 0.363
[2, 900] loss: 0.345
[3, 100] loss: 0.320
[3, 200] loss: 0.338
[3, 300] loss: 0.331
[3, 400] loss: 0.325
[3, 500] loss: 0.324
[3, 600] loss: 0.335
[3, 700] loss: 0.307
[3, 800] loss: 0.314
[3, 900] loss: 0.318
[4, 100] loss: 0.304
[4, 200] loss: 0.296
[4, 300] loss: 0.305
[4, 400] loss: 0.298
[4, 500] loss: 0.284
[4, 600] loss: 0.304
[4, 700] loss: 0.306
[4, 800] loss: 0.299
[4, 900] loss: 0.285
[5, 100] loss: 0.277
[5, 200] loss: 0.288
[5, 300] loss: 0.292
[5, 400] loss: 0.285
[5, 500] loss: 0.278
[5, 600] loss: 0.272
[5, 700] loss: 0.271
[5, 800] loss: 0.253
[5, 900] loss: 0.266
Finished Training


#### Ejemplo CIFAR-10 en CPU

In [14]:
device

device(type='cuda', index=0)

In [15]:
# Check if a GPU is available
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = "cpu"

# Define a simple convolutional neural network
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = x.view(-1, 64 * 16 * 16)  # Flatten the input
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Load the CIFAR-10 dataset and apply transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# Create an instance of the neural network
net = SimpleCNN()
# net.to(device)  # Move the network to the GPU

# Define a loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

# Record the starting time
start_time = time.time()

# Training loop
for epoch in range(5):  # You can increase the number of epochs for better results
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:  # Print every 100 mini-batches
            print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 100:.3f}")
            running_loss = 0.0

# Load the CIFAR-10 test dataset (or your validation dataset)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

# Evaluate the model on the test dataset
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Calculate and print the accuracy
accuracy = 100 * correct / total
print(f"Accuracy on the test dataset: {accuracy:.2f}%")

# Record the ending time
end_time = time.time()

# Calculate and print the elapsed time
elapsed_time = end_time - start_time
print(f"Training elapsed time: {elapsed_time:.2f} seconds")


print("Finished Training")

# Save the trained model
# torch.save(net.state_dict(), 'simple_cnn.pth')


Files already downloaded and verified
[1, 100] loss: 1.939
[1, 200] loss: 1.629
[1, 300] loss: 1.499
[1, 400] loss: 1.401
[1, 500] loss: 1.364
[1, 600] loss: 1.319
[1, 700] loss: 1.272
[2, 100] loss: 1.146
[2, 200] loss: 1.135
[2, 300] loss: 1.113
[2, 400] loss: 1.106
[2, 500] loss: 1.087
[2, 600] loss: 1.033
[2, 700] loss: 1.053
[3, 100] loss: 0.960
[3, 200] loss: 0.894
[3, 300] loss: 0.895
[3, 400] loss: 0.920
[3, 500] loss: 0.921
[3, 600] loss: 0.915
[3, 700] loss: 0.910
[4, 100] loss: 0.775
[4, 200] loss: 0.748
[4, 300] loss: 0.809
[4, 400] loss: 0.799
[4, 500] loss: 0.809
[4, 600] loss: 0.803
[4, 700] loss: 0.799
[5, 100] loss: 0.675
[5, 200] loss: 0.644
[5, 300] loss: 0.680
[5, 400] loss: 0.674
[5, 500] loss: 0.648
[5, 600] loss: 0.674
[5, 700] loss: 0.729
Files already downloaded and verified
Accuracy on the test dataset: 66.91%
Training elapsed time: 131.72 seconds
Finished Training


#### Ejemplo CIFAR-10 en GPU

In [16]:
# Check if a GPU is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Define a simple convolutional neural network
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = x.view(-1, 64 * 16 * 16)  # Flatten the input
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Load the CIFAR-10 dataset and apply transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# Create an instance of the neural network
net = SimpleCNN()
net.to(device)  # Move the network to the GPU

# Define a loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

# Record the starting time
start_time = time.time()

# Training loop
for epoch in range(5):  # You can increase the number of epochs for better results
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:  # Print every 100 mini-batches
            print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 100:.3f}")
            running_loss = 0.0

# Load the CIFAR-10 test dataset (or your validation dataset)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

# Evaluate the model on the test dataset
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Calculate and print the accuracy
accuracy = 100 * correct / total
print(f"Accuracy on the test dataset: {accuracy:.2f}%")

# Record the ending time
end_time = time.time()

# Calculate and print the elapsed time
elapsed_time = end_time - start_time
print(f"Training elapsed time: {elapsed_time:.2f} seconds")


print("Finished Training")

# Save the trained model
# torch.save(net.state_dict(), 'simple_cnn.pth')


Files already downloaded and verified
[1, 100] loss: 1.967
[1, 200] loss: 1.610
[1, 300] loss: 1.468
[1, 400] loss: 1.363
[1, 500] loss: 1.350
[1, 600] loss: 1.288
[1, 700] loss: 1.253
[2, 100] loss: 1.162
[2, 200] loss: 1.126
[2, 300] loss: 1.091
[2, 400] loss: 1.093
[2, 500] loss: 1.082
[2, 600] loss: 1.040
[2, 700] loss: 1.028
[3, 100] loss: 0.893
[3, 200] loss: 0.899
[3, 300] loss: 0.910
[3, 400] loss: 0.916
[3, 500] loss: 0.916
[3, 600] loss: 0.912
[3, 700] loss: 0.883
[4, 100] loss: 0.771
[4, 200] loss: 0.758
[4, 300] loss: 0.740
[4, 400] loss: 0.778
[4, 500] loss: 0.785
[4, 600] loss: 0.786
[4, 700] loss: 0.791
[5, 100] loss: 0.628
[5, 200] loss: 0.659
[5, 300] loss: 0.681
[5, 400] loss: 0.645
[5, 500] loss: 0.654
[5, 600] loss: 0.696
[5, 700] loss: 0.681
Files already downloaded and verified
Accuracy on the test dataset: 65.92%
Training elapsed time: 30.37 seconds
Finished Training
